In [15]:
import sys
sys.path.append('../src')
import TCT as TCT
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [16]:
# Step1: List all the APIs in the translator system
APInames = TCT.list_Translator_APIs()
print(len(APInames))
#print(APInames.keys())

65


In [17]:
# Step 2: Get metaKG from Translator APIs
metaKG = TCT.get_KP_metadata(APInames) # This only applies to the Translator APIs
print(metaKG.columns)
print(metaKG.shape)

#metaKG.to_csv('../metaData/metaKG.csv', index=False)
metaKG = pd.read_csv('../metaData/metaKG.csv')
All_predicates = list(set(metaKG['KG_category']))

In [18]:
query_json_temp = {
        "message": {
            "query_graph": {
                "nodes": {
                    "n0": {
                        "ids":[],
                        "categories":["biolink:category"]
                    },
                    "n1": {
                        "categories":["biolink:category"]
                }
                },
                "edges": {
                    "e1": {
                        "subject": "n0",
                        "object": "n1",
                        "predicates": ["biolink:predicates"]
                    }
                }
            }
        }
    }

In [19]:
KG_category = list(set( list(metaKG['Subject'].unique()) + list(metaKG['Object'].unique())))

KG_predicates = list(metaKG['KG_category'].unique())


In [20]:
question1 = "What genes or proteins interact with gene KRAS?"  #success

question = question1


In [21]:
query_json = str(query_json_temp)

ALL_predicates = ','.join(list(set(KG_predicates)))

ALL_categories = ','.join(list(set(KG_category)))

input_text = "We know the avaiable predicates in the KG are: " + ALL_predicates + ". We also know the avaiable categories in the KGs are "+ ALL_categories +". We also know a TRAPI message temple is " +query_json+ ". With the question of " + question + " What is the json format of message to represent this question? The following rules for the output: 1) The result must be just a json format with the same structure with template; 2)categories should be replaced from the categories in the KG; 3)predicts can be replaced from the predicates in the KG; 4) can use the name to fill the ids; 4)the output must start with '{' and end with '}', and be a standard json format.  At least one ids should be given and No annotations are needed!"


In [22]:
import openai
import json
confi = json.loads(open("confidential_key.json").read())
openai.organization = confi['Organization']
openai.api_key = confi['API_key']


In [23]:
query_json_byGPT = TCT.query_chatGPT(input_text)

query_json_byGPT

20


'{"message": {"query_graph": {"nodes": {"n0": {"ids": ["KRAS"], "categories": ["biolink:Gene"]}, "n1": {"categories": ["biolink:Protein", "biolink:Gene"]}}, "edges": {"e1": {"subject": "n0", "object": "n1", "predicates": ["biolink:interacts_with"]}}}}}'